In [1]:
from qnlib.compiling import CircuitDecomposer, RandomCompiling
from qcal.interface.superstaq.transpiler import CirqTranspiler, transpilation_error
from qcal.gate.single_qubit import Rz, X90, Id
from collections import defaultdict
import cirq

In [2]:
measurement_circuit = cirq.Circuit()
qutrit = cirq.LineQid(0, dimension = 3)
measurement_circuit.append(cirq.measure(qutrit, key="x"))

cycles = 100
circuits = []
corrections = []

for _ in range(cycles):
    # Initialize with measurement_twirl=True
    compiler = RandomCompiling(measurement_circuit, d=3, measurement_twirl=True)
    compiled_circuit = compiler.compile()

    decomposer = CircuitDecomposer(compiled_circuit)
    decomposed_circuit = decomposer.decompose_circuit(num_qudits=1, dim=3, native_gates='RzRx')
    decomposed_circuit.append(cirq.measure(qutrit, key="x"))

    # After compiling the circuit, get the corrections:
    corrections.append(compiler.get_measurement_corrections())
    circuits.append(decomposed_circuit)

post_procesing = zip(circuits, corrections)

In [3]:
#I.e.
print(circuits[0])

for moment in circuits[0]:
    for op in moment:
        print(type(op.gate).__name__)

0 (d=3): ───Rz───Rz───M('x')───
Rz
Rz
MeasurementGate


In [4]:
gate_mapper = defaultdict(lambda: transpilation_error,
                {'Id': Id,
                 'Rz': Rz,
                 'X90': X90,
                }
            )

In [5]:
transpiler = CirqTranspiler(gate_mapper)
transpiled_circuits = transpiler.transpile(circuits=circuits)

In [6]:
print(transpiled_circuits[0:25])

                                              circuit
0     (Cycle Rz:(0,), Cycle Rz:(0,), Cycle Meas:(0,))
1     (Cycle Rz:(0,), Cycle Rz:(0,), Cycle Meas:(0,))
2   (Cycle X90:(0,), Cycle X90:(0,), Cycle X90:(0,...
3   (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
4   (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
5   (Cycle Rz:(0,), Cycle Rz:(0,), Cycle X90:(0,),...
6                     (Cycle I:(0,), Cycle Meas:(0,))
7   (Cycle Rz:(0,), Cycle Rz:(0,), Cycle X90:(0,),...
8   (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
9   (Cycle X90:(0,), Cycle X90:(0,), Cycle X90:(0,...
10  (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
11  (Cycle Rz:(0,), Cycle Rz:(0,), Cycle X90:(0,),...
12  (Cycle Rz:(0,), Cycle Rz:(0,), Cycle X90:(0,),...
13    (Cycle Rz:(0,), Cycle Rz:(0,), Cycle Meas:(0,))
14    (Cycle Rz:(0,), Cycle Rz:(0,), Cycle Meas:(0,))
15  (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
16  (Cycle X90:(0,), Cycle X90:(0,), Cycle X90:(0,...
17    (Cycle Rz:(0,), Cycle 